In [61]:
from bs4 import BeautifulSoup
import urllib
import re 
import numpy as np
import json
from collections import OrderedDict

In [62]:
page = urllib.request.urlopen('http://www.imdb.com/list/ls036115103/').read()

In [63]:
soup = BeautifulSoup(page)
print(type(soup))

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file /Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


<class 'bs4.BeautifulSoup'>


In [64]:
characters_ls_even = soup.find_all("div", class_="list_item even")
characters_ls_odd = soup.find_all("div", class_="list_item odd")
chars_odd_dict = {}
chars_even_dict = {}
chars_odd_dict = OrderedDict(chars_odd_dict)
chars_even_dict = OrderedDict(chars_even_dict)

In [65]:
#print(characters_ls_even[0])
#Class - number, secondary, description

In [66]:
characters_ls_odd[0]

<div class="list_item odd">
<div class="image"> <a href="/character/ch0556868/"><div class="hover-over-image zero-z-index" data-const="ch0556868">
<img alt="Image of Tony Stark" class="zero-z-index" height="209" src="https://images-na.ssl-images-amazon.com/images/M/MV5BZmE1YjFkNGQtMjYyNC00NmFjLTkyNjQtNDMxYzc5ZTQ4MGZhXkEyXkFqcGdeQXVyNjczMDY1MzM@._V1._SY209_CR87,0,140,209_.jpg" width="140"/>
</div>
</a>
</div>
<div class="number">1.</div>
<div class="info">
<b><a href="/character/ch0556868/">Tony Stark</a></b>
<div class="secondary"><a href="/title/tt0371746/?ref_=rlm">Iron Man</a> <span class="year_type">(2008)</span>, <a href="/name/nm0000375/?ref_=rlm">Robert Downey Jr.</a></div>
</div>
<div class="description"><span class="bqstart">“ </span>played by Robert Downey Jr.<br/><br/>* IRON MAN &lt;77:15&gt;<br/>* THE INCREDIBLE HULK &lt;:30&gt;<br/>* IRON MAN 2 &lt;61:15&gt;<br/>* THE AVENGERS &lt;32:15&gt;<br/>* IRON MAN 3 &lt;62:15&gt;<br/>* AVENGERS: AGE OF ULTRON &lt;27:15&gt;<br/>* CA

In [67]:
for character in characters_ls_odd:
    chars_odd_dict[character.b.get_text()] = character.find("div", {"class": "description"}).get_text()
for character in characters_ls_even:
    chars_even_dict[character.b.get_text()] = character.find("div", {"class": "description"}).get_text()

In [68]:
characters_ls_odd[1].find("div", {"class": "description"}).get_text()

'“\xa0played by Chris Hemsworth* THOR <43:15>* THE AVENGERS <18>* THOR: THE DARK WORLD <35>* AVENGERS: AGE OF ULTRON <13:15>* DOCTOR STRANGE <:45>* THOR: RAGNAROK <57>167:15 minutes\xa0” - ninewheels0'

In [69]:
#Implementing a regex filter to filter out \xa0
#txt = chars_even_dict['Agent Phil Coulson']
def get_info(txt):
    txt = re.subn(pattern=r'\xa0', repl='', string=txt)[0]
    txt = re.subn(pattern=r'ninewheels0', repl='', string=txt)[0]
    txt = re.subn(pattern=r' - ', repl='', string=txt)[0]
    txt = re.subn(pattern=r'[“”]', repl='', string=txt)[0] #“
    txt = re.subn(pattern=r'\d*:?\d*? minutes', repl='', string=txt)[0]
    
    #try:
    #    txt = txt.split('*')
    #    temp = {}
    #    temp = OrderedDict(temp)
    #    temp['actor'] = txt[0].split('played by ')[1]
    #    
    #    for i in range(1, len(txt)):
    #        movie_and_duration = txt[i].replace(' ', '', 1).replace('>', '').split(' <')
    #        temp['{0}'.format(movie_and_duration[0].replace('', ''))] = movie_and_duration[1]
    #    return temp
    #except:
    #    return txt
    
    txt = txt.split('*')
    temp = {}
    temp = OrderedDict(temp)
    try: 
        temp['actor'] = txt[0].split('played by ')[1]
    except:
        temp['actor'] = txt[0].split('played ')[1]
    for i in range(1, len(txt)):
        movie_and_duration = txt[i].replace(' ', '', 1).replace('>', '').split(' <')
        temp['{0}'.format(movie_and_duration[0].replace('', ''))] = movie_and_duration[1]
    
    return temp
        
        
    
    

Dictionary of the form:
    {character_Name: {actor: actor_name, 
                      movie_1: time_movie_1, 
                      movie_2: time_movie_2}
    
    }

In [70]:
tp = dict(chars_odd_dict)

In [71]:
for ch in tp.keys():
    tp[ch] = get_info(tp[ch])

In [72]:
for character in chars_odd_dict:
    chars_odd_dict[character] = get_info(chars_odd_dict[character])

for character in chars_even_dict:
    chars_even_dict[character] = get_info(chars_even_dict[character])

In [73]:
def merge(dict1,*dicts):
    for dict2 in dicts:
        dict1.update(dict2)
    return dict1

In [74]:
characters = merge(chars_odd_dict, chars_even_dict)

In [75]:
with open('characters.json', 'w') as file:
    json.dump(characters, file)

#Documentation:
<p>
https://www.crummy.com/software/BeautifulSoup/bs4/doc/ 
<br>
http://www.imdb.com/list/ls036115103/?start=1&view=detail&sort=listorian:asc
<br>
http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html
</p>
